# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NOTEBOOK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\NOTEBOOK\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NOTEBOOK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///nlp_messages.db')
    df = pd.read_sql('SELECT * FROM cat_messages', engine)
    X = df.message
    y = df.iloc[:,4:]
    return X, y

In [3]:
### 2. Write a tokenization function to process your text data

url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    tokens = word_tokenize(text)
    
    lemmatizer = WordNetLemmatizer()
    
    words = [lemmatizer.lemmatize(words).strip() for words in tokens if words not in stopwords.words("english")]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
         ('tfidf', TfidfTransformer()),
         ('clf', MultiOutputClassifier(RandomForestClassifier()))])
    
    parameters = {
         'vect__ngram_range': ((1, 1), (1, 2)),
        #'vect__max_df': (0.5, 0.75, 1.0),
        #'vect__max_features': (None, 5000, 10000),
        #'tfidf__use_idf': (True, False),
        #'clf__estimator__n_estimators': [50, 100, 200],
        #'clf__estimator__min_samples_split': [2, 3, 4]
    }
    
    cv = GridSearchCV(pipeline, param_grid = parameters, cv=2, verbose=3)
    
    return cv

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
def main():
    #load data
    X, y = load_data()
    
    #split data
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    
    #initialize model
    model = build_model()
    
    #train model
    model.fit(X_train, y_train)
    
    #predict y values
    y_pred = model.predict(X_test)
    
    #display f1 score, precision and recall for each output category
    display_results(y_test, y_pred)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
def display_results(y_test, y_pred):
    results = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])
    for i, cat in enumerate(y_test.columns):
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[cat], y_pred[:,i], average='weighted')
        results.at[i+1, 'Category'] = cat
        results.at[i+1, 'f_score'] = f_score
        results.at[i+1, 'precision'] = precision
        results.at[i+1, 'recall'] = recall
        
    print('Aggregated f_score:', results['f_score'].mean())
    print('Aggregated precision:', results['precision'].mean())
    print('Aggregated recall:', results['recall'].mean())
    return results

### 6. Improve your model
Use grid search to find better parameters. 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
#load data
X, y = load_data()
    
#split data
X_train, X_test, y_train, y_test = train_test_split(X,y)

model = build_model()
#train model
model.fit(X_train, y_train)
   
#predict y values
y_pred = model.predict(X_test)
    
#display f1 score, precision and recall for each output category
display_results(y_test, y_pred)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] vect__ngram_range=(1, 1) ........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............ vect__ngram_range=(1, 1), score=0.237, total= 7.9min
[CV] vect__ngram_range=(1, 1) ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.9min remaining:    0.0s


[CV] ............ vect__ngram_range=(1, 1), score=0.238, total= 7.7min
[CV] vect__ngram_range=(1, 2) ........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 15.7min remaining:    0.0s


[CV] ............ vect__ngram_range=(1, 2), score=0.237, total=10.4min
[CV] vect__ngram_range=(1, 2) ........................................
[CV] ............ vect__ngram_range=(1, 2), score=0.237, total=10.8min


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 36.9min finished
C:\Users\NOTEBOOK\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Aggregated f_score: 0.933231262538254
Aggregated precision: 0.9392822325022314
Aggregated recall: 0.9468221272844405


,Category,f_score,precision,recall
1,related,0.764945,0.789435,0.800122
2,request,0.882723,0.894135,0.896857
3,offer,0.992911,0.990563,0.99527
4,aid_related,0.75892,0.764586,0.764266
5,medical_help,0.893555,0.900625,0.923558
6,medical_products,0.929942,0.942361,0.95087
7,search_and_rescue,0.961385,0.971068,0.973299
8,security,0.975649,0.975699,0.983522
9,military,0.952223,0.955565,0.966738
10,child_alone,1,1,1


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [16]:
#Export model as pickle file
pickle.dump(model, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.